In [251]:
#雙點交配

p1=[1,2,3,4,5,6]
print(len(p1)+1)
p2=[7,8,9,10,11,12]
CutPoint=[]
import random
size=range(1,7)  #染色體大小 #10+10(range(1,21))

CutPoint=random.sample(size, 2)
CutPoint.sort() 
print(CutPoint)

strpoint=CutPoint[0]
endpoint=CutPoint[1]

c1=p1[:]
c2=p2[:]
c1[strpoint-1:endpoint]=p2[strpoint-1:endpoint]
c2[strpoint-1:endpoint]=p1[strpoint-1:endpoint]
print(c1,c2)


7
[5, 6]
[1, 2, 3, 4, 11, 12] [7, 8, 9, 10, 5, 6]


In [4]:
if(len(M1EndT)==0):
    



if(M1EndT[-1]>M2EndT[-1] and M1EndT[-1]>M3EndT[-1]):
    Makespan=M1EndT[-1]
elif(M2EndT[-1]>M1EndT[-1] and M2EndT[-1]>M3EndT[-1]):
    Makespan=M2EndT[-1]
else:
    Makespan=M3EndT[-1]


IndexError: list index out of range

In [9]:
a=0
if(a==2 or a==1 or a==2):
    print("YeS")



In [ ]:
#合成三維array

import numpy as np
a=np.zeros((5, 20))
list1=[0.2,0.8,0.4,0.9,0.4,0.1,0.7]
list2=[3,4,5,6]
for i in range(len(list1)):
    a[0][i]=list1[i]
for i in range(len(list2)):
    a[1][i]=list2[i]
print(a)

#取出非0的值
print(a[0])
x = a[0].nonzero() 
print(x)
print(a[0][x])

In [30]:
import numpy as np
a=np.zeros((10,5,20))
list1=[0.2,0.8,0.4,0.9,0.4,0.1,0.7]
list2=[3,4,5,6]
for i in range(len(list1)):
    a[0][0][i]=list1[i]
for i in range(len(list2)):
    a[0][1][i]=list2[i]
print(a)

[[[0.2 0.8 0.4 0.9 0.4 0.1 0.7 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [3.  4.  5.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]]

 [[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]]

 [[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [31]:
import numpy as np

aa = np.array([[[1,2,3],[4,5,6]],[[2,2,3],[4,5,6]],[[3,2,3],[4,5,6]]])
a = np.array([[4,2,3],[4,5,6]])
data = np.append(aa,a)#先拼接成一个行向量
print(data)

dim = aa.shape#获取原矩阵的维数
print('原矩阵维数：',dim)
data1 = data.reshape(dim[0]+1,dim[1],dim[2])#再通过原矩阵的维数重新组合

print('合并矩阵：\n',data1)
print('维数：',data1.shape)

[1 2 3 4 5 6 2 2 3 4 5 6 3 2 3 4 5 6 4 2 3 4 5 6]
原矩阵维数： (3, 2, 3)
合并矩阵：
 [[[1 2 3]
  [4 5 6]]

 [[2 2 3]
  [4 5 6]]

 [[3 2 3]
  [4 5 6]]

 [[4 2 3]
  [4 5 6]]]
维数： (4, 2, 3)


In [39]:
import math
a=math.ceil(2.5)
a

3

In [91]:
#任兩條進行交配
import random
PopulationNum=5
size=range(1,6)  #母體大小 

AnyTwo=random.sample(size, 2)
AnyTwo.sort()
print(AnyTwo)

FirstC=AnyTwo[0]
SecondC=AnyTwo[1]
print(FirstC-1,SecondC-1)

[1, 4]
0 3


In [138]:
import numpy as np
A=np.array([[[8,2],[3,7],[5,0]],[[2,5],[8,3],[10,0]]])
print(A) 
print("--------------")
Answer=np.array([[[2,5],[8,3],[10,0]],[[8,2],[3,7],[5,0]]])
print(Answer)


[[[ 8  2]
  [ 3  7]
  [ 5  0]]

 [[ 2  5]
  [ 8  3]
  [10  0]]]
--------------
[[[ 2  5]
  [ 8  3]
  [10  0]]

 [[ 8  2]
  [ 3  7]
  [ 5  0]]]


In [140]:
for i in range(0,2):
    print(i)
for i in range(2,5):
    print(i)

0
1
2
3
4


In [147]:
    PopulationNum=50
    size=range(0,PopulationNum)  #母體大小
    for i in range(50):
        AnyTwo=random.sample(size, 1)
        print(AnyTwo)
    

[20]
[49]
[33]
[41]
[27]
[7]
[34]
[33]
[28]
[42]
[40]
[43]
[24]
[48]
[24]
[46]
[1]
[38]
[19]
[34]
[48]
[13]
[10]
[11]
[0]
[1]
[32]
[40]
[31]
[0]
[47]
[3]
[38]
[24]
[9]
[27]
[39]
[2]
[30]
[26]
[48]
[35]
[38]
[0]
[8]
[41]
[4]
[30]
[1]
[21]


In [154]:
EndTList=[]
EndTList.append(M1EndT[-1])
EndTList.append(M2EndT[-1])
EndTList.append(M3EndT[-1])

OrderEndT=sorted(EndTList,reverse=True) #由大到小
print(OrderEndT[0])


[80, 90, 0]
[90, 80, 0]
90


In [157]:
a=3
sequence=[i for i in range(a)]
sequence

[0, 1, 2]

In [67]:
#菁英制
PopulationNum=10
#for i in range(int(PopulationNum*0.2)): #10
    #print(i)
    #FinalChromosome[i]=TotalChromosome[OrderMs[i][0]]

#等級輪盤制
#for i in range(int(PopulationNum*0.2),PopulationNum*2): #10-100
    #print(i)

makespan=[8,9,10,11,12,13,14,15]

def sum(num):
    sum = 0
    x=1
    while x < num+1:
        sum = sum + x
        x+=1
    return sum
Sum=sum(len(makespan))
t=0
proba_list=[0]
for i in range(len(makespan)-1):
    proba=(len(makespan)-i)/Sum
    t+=proba
    proba_list.append(t)
print(proba_list)

def getk2():
    import random
    selectone=random.random()
    #print(selectone)

    k2=-1
    for i in range(len(proba_list)):
        if(selectone>proba_list[i]):
            k2+=1
    return k2

a=[]
count=0
while(len(a)<7):
    count+=1
    temp=getk2()
    if(temp not in a):
        a.append(temp)
print(count)
print(a)

    


[0, 0.2222222222222222, 0.41666666666666663, 0.5833333333333333, 0.7222222222222221, 0.8333333333333333, 0.9166666666666666, 0.9722222222222222]
18
[0, 3, 4, 6, 1, 5, 2]


In [53]:
a={1,2,3}
b={4,5,6}
set(a).intersection(set(b)

SyntaxError: unexpected EOF while parsing (<ipython-input-53-0266af91a7a8>, line 3)

In [54]:
a=1
b=[]
a in b

for i in range(10):
    if(a in b):
      
        b.append(i)



False